# تدريب وحفظ واختبار النماذج

تدريب وحفظ واختبار جميع النماذج المستخدمة في المشروع

## استيراد المكتبات المطلوبة

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import KMeans
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import joblib
import logging
import os
from datetime import datetime

# تعيين مستوى التسجيل
logging.basicConfig(level=logging.INFO)

## تحميل وتحضير البيانات

In [2]:
# تحميل البيانات المعالجة
df = pd.read_csv('../data/processed/sales_data_processed_20250122_052318.csv')
print('أبعاد البيانات:', df.shape)

أبعاد البيانات: (3000, 12)


In [3]:
# تحضير الميزات
def prepare_features(df):
    # تشفير المتغيرات الفئوية
    le = LabelEncoder()
    categorical_cols = ['Category', 'Region', 'Segment']
    encoded_cols = {}
    for col in categorical_cols:
        df[f'{col}_Encoded'] = le.fit_transform(df[col])
        encoded_cols[col] = le
    
    # تطبيع المتغيرات العددية
    scaler = StandardScaler()
    numeric_cols = ['Quantity', 'Sales', 'Discount', 'Profit']
    scaled_cols = {}
    for col in numeric_cols:
        df[f'{col}_Scaled'] = scaler.fit_transform(df[[col]])
        scaled_cols[col] = scaler
    
    return df, encoded_cols, scaled_cols

# تحضير البيانات
df, encoders, scalers = prepare_features(df)
print('تم تحضير الميزات بنجاح')

تم تحضير الميزات بنجاح


## تدريب النماذج

In [4]:
# تدريب نموذج شجرة القرار
def train_decision_tree(df):
    features = ['Year', 'Month', 'Quarter', 'Quantity_Scaled', 'Discount_Scaled',
                'Category_Encoded', 'Region_Encoded', 'Segment_Encoded']
    
    X = df[features]
    y = df['Sales_Category']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    dt = DecisionTreeClassifier(random_state=42, max_depth=5)
    dt.fit(X_train, y_train)
    
    accuracy = dt.score(X_test, y_test)
    print(f'دقة نموذج شجرة القرار: {accuracy:.2f}')
    
    return dt, (X_test, y_test)

# تدريب النموذج
dt_model, dt_test_data = train_decision_tree(df)

دقة نموذج شجرة القرار: 0.32


In [5]:
# تدريب نموذج Naive Bayes
def train_naive_bayes(df):
    features = ['Year', 'Month', 'Quarter', 'Quantity_Scaled', 'Discount_Scaled',
                'Category_Encoded', 'Region_Encoded', 'Segment_Encoded']
    
    X = df[features]
    y = df['Sales_Category']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    nb = GaussianNB()
    nb.fit(X_train, y_train)
    
    accuracy = nb.score(X_test, y_test)
    print(f'دقة نموذج Naive Bayes: {accuracy:.2f}')
    
    return nb, (X_test, y_test)

# تدريب النموذج
nb_model, nb_test_data = train_naive_bayes(df)

دقة نموذج Naive Bayes: 0.30


In [6]:
# تدريب نموذج K-means
def train_kmeans(df):
    features = ['Quantity_Scaled', 'Sales_Scaled', 'Discount_Scaled', 'Profit_Scaled']
    X = df[features]
    
    kmeans = KMeans(n_clusters=3, random_state=42)
    kmeans.fit(X)
    
    inertia = kmeans.inertia_
    print(f'متوسط مسافة النقاط عن مراكزها: {inertia:.2f}')
    
    return kmeans, X

# تدريب النموذج
kmeans_model, kmeans_data = train_kmeans(df)

d:\python\data_mining_project\.venv\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
d:\python\data_mining_project\.venv\lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "d:\python\data_mining_project\.venv\lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


متوسط مسافة النقاط عن مراكزها: 8278.80


In [7]:
# تدريب نموذج Apriori
def train_apriori(df):
    # إنشاء جدول المعاملات
    transactions = df.groupby(['Order_Date'])[['Category', 'Region', 'Segment']].agg(lambda x: list(x))
    
    # تحويل البيانات إلى تنسيق ثنائي
    encoded_vals = []
    for index, row in transactions.iterrows():
        rowset = []
        for item in row:
            rowset.extend(item)
        encoded_vals.append(rowset)
    
    # إنشاء DataFrame مشفر
    ohe = pd.get_dummies(pd.DataFrame(encoded_vals))
    
    # إيجاد مجموعات العناصر المتكررة
    frequent_itemsets = apriori(ohe, min_support=0.01, use_colnames=True)
    
    # استخراج قواعد الارتباط
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.3)
    
    print(f'تم استخراج {len(rules)} قاعدة ارتباط')
    
    return rules

# تدريب النموذج
apriori_rules = train_apriori(df)

تم استخراج 111 قاعدة ارتباط


## حفظ النماذج

In [8]:
def save_models(models, encoders, scalers):
    # إنشاء مجلد للنماذج
    os.makedirs('../models', exist_ok=True)
    
    # حفظ النماذج
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    for name, model in models.items():
        model_path = f'../models/{name}_{timestamp}.joblib'
        joblib.dump(model, model_path)
        print(f'تم حفظ نموذج {name} في {model_path}')
    
    # حفظ المشفرات
    encoder_path = f'../models/encoders_{timestamp}.joblib'
    joblib.dump(encoders, encoder_path)
    print(f'تم حفظ المشفرات في {encoder_path}')
    
    # حفظ المطبعات
    scaler_path = f'../models/scalers_{timestamp}.joblib'
    joblib.dump(scalers, scaler_path)
    print(f'تم حفظ المطبعات في {scaler_path}')

# تجميع النماذج
models = {
    'decision_tree': dt_model,
    'naive_bayes': nb_model,
    'kmeans': kmeans_model,
    'apriori': apriori_rules
}

# حفظ النماذج
save_models(models, encoders, scalers)

تم حفظ نموذج decision_tree في ../models/decision_tree_20250122_052900.joblib
تم حفظ نموذج naive_bayes في ../models/naive_bayes_20250122_052900.joblib
تم حفظ نموذج kmeans في ../models/kmeans_20250122_052900.joblib
تم حفظ نموذج apriori في ../models/apriori_20250122_052900.joblib
تم حفظ المشفرات في ../models/encoders_20250122_052900.joblib
تم حفظ المطبعات في ../models/scalers_20250122_052900.joblib


## اختبار النماذج

In [9]:
def test_models(models, test_data):
    results = {}
    
    # اختبار شجرة القرار
    if 'decision_tree' in models and 'decision_tree' in test_data:
        dt = models['decision_tree']
        X_test, y_test = test_data['decision_tree']
        dt_accuracy = dt.score(X_test, y_test)
        results['decision_tree'] = dt_accuracy
    
    # اختبار Naive Bayes
    if 'naive_bayes' in models and 'naive_bayes' in test_data:
        nb = models['naive_bayes']
        X_test, y_test = test_data['naive_bayes']
        nb_accuracy = nb.score(X_test, y_test)
        results['naive_bayes'] = nb_accuracy
    
    # اختبار K-means
    if 'kmeans' in models and 'kmeans' in test_data:
        kmeans = models['kmeans']
        X = test_data['kmeans']
        inertia = kmeans.inertia_
        results['kmeans'] = inertia
    
    # حفظ نتائج الاختبار
    os.makedirs('../results', exist_ok=True)
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    results_file = f'../results/model_test_results_{timestamp}.txt'
    
    with open(results_file, 'w', encoding='utf-8') as f:
        f.write("نتائج اختبار النماذج\n")
        f.write("-" * 50 + "\n\n")
        for model_name, score in results.items():
            f.write(f"{model_name}: {score:.4f}\n")
    
    print(f'تم حفظ نتائج الاختبار في {results_file}')
    return results

# تجميع بيانات الاختبار
test_data = {
    'decision_tree': dt_test_data,
    'naive_bayes': nb_test_data,
    'kmeans': kmeans_data
}

# اختبار النماذج
results = test_models(models, test_data)

# عرض النتائج
print('\nنتائج الاختبار:')
for model_name, score in results.items():
    print(f'{model_name}: {score:.4f}')

تم حفظ نتائج الاختبار في ../results/model_test_results_20250122_052900.txt

نتائج الاختبار:
decision_tree: 0.3183
naive_bayes: 0.2967
kmeans: 8278.7980
